<a href="https://colab.research.google.com/github/Subhashree2016/etl_llm_thesis/blob/main/BERT_ModelSchemaPredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install transformers library
!pip install transformers
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import io
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/adventureworks_tableschema.csv")  # Adjust the path as per your file location
df.head()

,Table Name,Column Name,Data Type,Constraint Name,Constraint Type
0,pg_default_acl,defaclrole,oid,pg_default_acl_role_nsp_obj_index,u
1,pg_default_acl,defaclnamespace,oid,pg_default_acl_role_nsp_obj_index,u
2,pg_default_acl,defaclobjtype,"""char""",pg_default_acl_role_nsp_obj_index,u
3,pg_default_acl,oid,oid,pg_default_acl_oid_index,p
4,pg_tablespace,oid,oid,pg_tablespace_oid_index,p


In [5]:
import pandas as pd

# Load the CSV file
df_m = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/adventureworks_tableschema.csv")

# Define a function to map similar data types
def map_data_type(data_type):
    if 'character' in data_type:
        return 'char'
    elif 'numeric' in data_type or 'real' in data_type:
        return 'numeric'
    elif 'smallint' in data_type or 'integer' in data_type:
        return 'integer'
    else:
        return data_type

# Apply the function to map data types
df_m ['Data Type'] = df_m ['Data Type'].apply(map_data_type)

# Replace 'char' and 'char' with 'char'
df_m ['Data Type'] = df_m ['Data Type'].replace({'char ': 'char', '"char"': 'char'})

# Save the modified DataFrame back to CSV
df_m .to_csv("modified_file.csv", index=False)

df_m

,Table Name,Column Name,Data Type,Constraint Name,Constraint Type
0,pg_default_acl,defaclrole,oid,pg_default_acl_role_nsp_obj_index,u
1,pg_default_acl,defaclnamespace,oid,pg_default_acl_role_nsp_obj_index,u
2,pg_default_acl,defaclobjtype,char,pg_default_acl_role_nsp_obj_index,u
3,pg_default_acl,oid,oid,pg_default_acl_oid_index,p
4,pg_tablespace,oid,oid,pg_tablespace_oid_index,p
...,...,...,...,...,...
473,sales.specialofferproduct,productid,integer,PK_SpecialOfferProduct_SpecialOfferID_ProductID,p
474,sales.specialofferproduct,productid,integer,FK_SpecialOfferProduct_Product_ProductID,f
475,sales.store,businessentityid,integer,PK_Store_BusinessEntityID,p
476,sales.store,businessentityid,integer,FK_Store_BusinessEntity_BusinessEntityID,f


In [6]:
import pandas as pd

# Load the modified CSV file
df_mod = pd.read_csv("modified_file.csv")

# Define the mapping for constraint types
constraint_types = {'c': 0, 'f': 1, 'p': 2, 'u': 3}
df_mod['Constraint_Type_Label'] = df_mod['Constraint Type'].map(constraint_types)

# Get distinct data types
distinct_data_types = df_mod['Data Type'].unique()

# Create a mapping from data type to label
data_type_to_label = {data_type: label for label, data_type in enumerate(distinct_data_types)}

# Add a new column for labels
df_mod['Datatype_Label'] = df_mod['Data Type'].map(data_type_to_label)

# Display the DataFrame with labels
#print(df)
distinct_data_types = df_mod['Datatype_Label'].unique()
print(distinct_data_types)
df_mod


[ 0  1  2  3  4  5  6  7  8  9 10 11 12]


,Table Name,Column Name,Data Type,Constraint Name,Constraint Type,Constraint_Type_Label,Datatype_Label
0,pg_default_acl,defaclrole,oid,pg_default_acl_role_nsp_obj_index,u,3,0
1,pg_default_acl,defaclnamespace,oid,pg_default_acl_role_nsp_obj_index,u,3,0
2,pg_default_acl,defaclobjtype,char,pg_default_acl_role_nsp_obj_index,u,3,1
3,pg_default_acl,oid,oid,pg_default_acl_oid_index,p,2,0
4,pg_tablespace,oid,oid,pg_tablespace_oid_index,p,2,0
...,...,...,...,...,...,...,...
473,sales.specialofferproduct,productid,integer,PK_SpecialOfferProduct_SpecialOfferID_ProductID,p,2,3
474,sales.specialofferproduct,productid,integer,FK_SpecialOfferProduct_Product_ProductID,f,1,3
475,sales.store,businessentityid,integer,PK_Store_BusinessEntityID,p,2,3
476,sales.store,businessentityid,integer,FK_Store_BusinessEntity_BusinessEntityID,f,1,3


# Code Summary ({table_name}: {column_text})

The code trains a BERT-based sequence classification model to predict multiple labels (number of columns, constraint type, and data type) for input text.

## Steps:

1. **Load Dataset:**
   - The code loads a schema dataset from a CSV file using pandas. It defines mappings for constraint types and data types.

2. **Initialize BERT Model and Tokenizer:**
   - It loads a pre-trained BERT model and tokenizer from the Hugging Face transformers library.

3. **Define Model Output Layers:**
   - The code defines output layers for predicting the number of columns, constraint type, and data type.

4. **Define Optimizer and Loss Function:**
   - It defines an optimizer (AdamW) and a cross-entropy loss function for training the model.

5. **Prepare Training Dataset:**
   - It prepares the training dataset by tokenizing input texts and converting labels to tensors.

6. **Define Data Loader:**
   - The training dataset is wrapped in a PyTorch DataLoader for batching and shuffling during training.

7. **Train the Model:**
   - The code trains the model for a specified number of epochs.
   - It iterates through batches of data, performs forward pass, calculates loss, and backpropagates to update model parameters.
   - Accuracy is calculated based on correctly predicted labels.
   - Loss and accuracy are printed after each epoch.

## Notes:
- It utilizes the BERT model architecture for sequence classification tasks.
- Tokenization using the input schema information (table and column names)
- PyTorch is used for model training and processing.
- The code is designed to handle multi-label classification tasks.


In [28]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import os

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the schema dataset from the CSV file
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/adventureworks_tableschema.csv")

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define the mapping for constraint types
constraint_types = {'c': 0, 'f': 1, 'p': 2, 'u': 3}

# Define the mapping for data types
data_type_mapping = {
    'oid': 0,
    '"char"': 1,
    'name': 2,
    'integer': 3,
    'oidvector': 4,
    'regproc': 5,
    'boolean': 6,
    'numeric': 7,
    'text': 8,
    'char': 1,  # Consider both "char" and "char" as the same
    '"Phone"': 9,
    'date': 10,
    'timestamp without time zone': 11,
    'uuid': 12
}
default_data_type_label = 13  # Default value for unknown data types

# Load pre-trained BERT model and tokenizer
max_length = 128
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
num_labels = len(constraint_types)
num_columns_output_size = 2
hidden_size = 768
num_constraint_types = len(constraint_types)
constraint_type_output_size = num_constraint_types
num_data_types = len(data_type_mapping)
data_type_output_size = num_data_types
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, output_hidden_states=True).to(device)

# Define output layers
num_columns_output_layer = torch.nn.Linear(hidden_size, num_columns_output_size)
constraint_type_output_layer = torch.nn.Linear(hidden_size, constraint_type_output_size)
data_type_output_layer = torch.nn.Linear(hidden_size, data_type_output_size)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Define training dataset
texts = []
num_columns_labels = []
constraint_type_labels = []
data_type_labels = []

for _, row in train_df.iterrows():  # Use train_df instead of df
    table_name = row['Table Name']
    columns_text = ': '.join([f"{col}: {row[col]}" for col in row.index if col != 'Table Name'])
    input_text = f"{table_name}: {columns_text}"
    texts.append(input_text)

    # Map constraint type to its corresponding integer label
    constraint_type_label = constraint_types[row['Constraint Type']]
    constraint_type_labels.append(constraint_type_label)

    # Number of columns label (1 or 0)
    num_columns_label = 1 if pd.notna(row['Column Name']) else 0
    num_columns_labels.append(num_columns_label)

    # Map data type to its corresponding integer label
    data_type_label = data_type_mapping.get(row['Data Type'], default_data_type_label)
    data_type_labels.append(data_type_label)

# Filter out the default data type label
filtered_texts = []
filtered_num_columns_labels = []
filtered_constraint_type_labels = []
filtered_data_type_labels = []

for i in range(len(data_type_labels)):
    if data_type_labels[i] != default_data_type_label:
        filtered_texts.append(texts[i])
        filtered_num_columns_labels.append(num_columns_labels[i])
        filtered_constraint_type_labels.append(constraint_type_labels[i])
        filtered_data_type_labels.append(data_type_labels[i])

# Update the lists with filtered values
texts = filtered_texts
num_columns_labels = filtered_num_columns_labels
constraint_type_labels = filtered_constraint_type_labels
data_type_labels = filtered_data_type_labels

# Tokenize input texts
train_encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

# Create PyTorch dataset
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'],
                              torch.tensor(num_columns_labels), torch.tensor(constraint_type_labels),
                              torch.tensor(data_type_labels))

# Define data loader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Train the model
model.to(device)

for epoch in range(3):  # Example: 3 epochs
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch in train_loader:
        input_ids, attention_mask, num_columns_labels, constraint_type_labels, data_type_labels = [data.to(device) for data in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.hidden_states[-1]  # Get the last hidden state from BERT model
        pooled_output = last_hidden_state[:, 0, :]  # Use the [CLS] token representation as the pooled output

        # Predict the number of columns
        num_columns_output = torch.sigmoid(num_columns_output_layer(pooled_output))
        predicted_num_columns = torch.argmax(num_columns_output, dim=1)

        # Predict the constraint type
        constraint_type_output = torch.softmax(constraint_type_output_layer(pooled_output), dim=1)
        predicted_constraint_types = torch.argmax(constraint_type_output, dim=1)

        # Predict the data type
        data_type_output = torch.softmax(data_type_output_layer(pooled_output), dim=1)
        predicted_data_types = torch.argmax(data_type_output, dim=1)

        # Calculate loss for each output
        num_columns_loss = loss_fn(num_columns_output.squeeze(-1), num_columns_labels)
        constraint_type_loss = loss_fn(constraint_type_output, constraint_type_labels)
        data_type_loss = loss_fn(data_type_output, data_type_labels)

        # Total loss
        total_loss = num_columns_loss + constraint_type_loss + data_type_loss

        # Calculate accuracy
        correct_predictions += ((predicted_num_columns == num_columns_labels) &
                                (predicted_constraint_types == constraint_type_labels) &
                                (predicted_data_types == data_type_labels)).sum().item()
        total_samples += input_ids.size(0)

        # Backpropagation
        total_loss.backward()
        optimizer.step()

    # Print loss and accuracy after each epoch
    epoch_loss = total_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples
    print(f'Epoch {epoch + 1}: Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

# Save the trained model
output_dir = "/content/drive/MyDrive/Colab Notebooks/Saved_Model_tablecol"
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("Model saved successfully at:", output_dir)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Loss: 0.1625, Accuracy: 0.3223
Epoch 2: Loss: 0.1576, Accuracy: 0.7319
Epoch 3: Loss: 0.1411, Accuracy: 0.7982
Model saved successfully at: /content/drive/MyDrive/Colab Notebooks/Saved_Model_tablecol


Prediction with data 'Department Table' (With header)

*** input_text = f"{table_name},column {i+1}: {value}"***

In [22]:
import os
import pandas as pd

# Specify the path to the CSV file
input_file_path = "/content/drive/MyDrive/Colab Notebooks/Data_subset_adventures_headers/department.csv"


# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your pre-trained BERT model and tokenizer
model_path = "/content/drive/MyDrive/Colab Notebooks/Saved_Model_tablecol"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path).to(device)

# Check if the input file is a CSV file
if input_file_path.endswith('.csv'):
    # Initialize a list to store the predictions
    predictions = []

    # Read the CSV file
    df = pd.read_csv(input_file_path)  # Read CSV without header
    table_name = os.path.splitext(os.path.basename(input_file_path))[0]  # Get table name from file name without extension

    # Iterate through each row in the CSV file
    for index, row in df.iterrows():
        # Iterate through each column in the row
        for i, value in enumerate(row):
            # Concatenate the value with the column number and table name
            input_text = f"{table_name},column {i+1}: {value}"
            print('input text:', input_text)

            # Tokenize input texts
            tokenized_data = tokenizer(input_text, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

            # Prepare input tensors
            input_ids = tokenized_data['input_ids'].to(device)
            attention_mask = tokenized_data['attention_mask'].to(device)

            # Perform inference
            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask)
                last_hidden_state = outputs.hidden_states[-1]
                pooled_output = last_hidden_state[:, 0, :]
                logits = outputs.logits
                num_columns_output = torch.sigmoid(num_columns_output_layer(pooled_output))
                constraint_type_output = torch.softmax(constraint_type_output_layer(pooled_output), dim=1)

                # Add prediction for data type
                data_type_output = data_type_output_layer(pooled_output)  # Assuming you have a separate output layer for data type
                predicted_data_type = torch.argmax(data_type_output, dim=1).cpu().numpy()

            # Post-process the output
            predicted_num_columns = torch.argmax(num_columns_output, dim=1).cpu().numpy()
            predicted_constraint_types = torch.argmax(constraint_type_output, dim=1).cpu().numpy()

            # Append the predictions to the list
            predictions.append({
                'Input Text': input_text,
                'Predicted Number of Columns': predicted_num_columns[0],
                'Predicted Constraint Type': predicted_constraint_types[0],
                'Predicted Data Type': predicted_data_type[0]  # Include the predicted data type
            })

    # Convert the list of dictionaries to a DataFrame
    predictions_df_H_dept = pd.DataFrame(predictions)

    # Save the predictions to a CSV file
    output_file_path = "/content/drive/MyDrive/Colab Notebooks/predictions_col_dept_H.csv"
    predictions_df_H_dept.to_csv(output_file_path, index=False)

else:
    print("Input file is not a CSV file.")


input text: department,column 1: 1
input text: department,column 2: Engineering
input text: department,column 3: Research and Development
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 2
input text: department,column 2: Tool Design
input text: department,column 3: Research and Development
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 3
input text: department,column 2: Sales
input text: department,column 3: Sales and Marketing
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 4
input text: department,column 2: Marketing
input text: department,column 3: Sales and Marketing
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 5
input text: department,column 2: Purchasing
input text: department,column 3: Inventory Management
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 6
input text: department,column 2: 

In [23]:
predictions_df_H_dept

,Input Text,Predicted Number of Columns,Predicted Constraint Type,Predicted Data Type
0,"department,column 1: 1",1,0,3
1,"department,column 2: Engineering",1,2,3
2,"department,column 3: Research and Development",1,2,3
3,"department,column 4: 2008-04-30 00:00:00",1,2,3
4,"department,column 1: 2",1,0,3
...,...,...,...,...
59,"department,column 4: 2008-04-30 00:00:00",1,2,3
60,"department,column 1: 16",1,2,3
61,"department,column 2: Executive",1,0,3
62,"department,column 3: Executive General and Adm...",1,2,3


Prediction with data 'Department Table' (Without header )

*** input_text = f"{table_name},column {i+1}: {value}"***

In [24]:
import os
import pandas as pd

# Specify the path to the CSV file
input_file_path =  "/content/drive/MyDrive/Colab Notebooks/data_subset_adventureworks/department.csv"


# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your pre-trained BERT model and tokenizer
model_path = "/content/drive/MyDrive/Colab Notebooks/Saved_Model_tablecol"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path).to(device)

# Check if the input file is a CSV file
if input_file_path.endswith('.csv'):
    # Initialize a list to store the predictions
    predictions = []

    # Read the CSV file
    df = pd.read_csv(input_file_path)  # Read CSV without header
    table_name = os.path.splitext(os.path.basename(input_file_path))[0]  # Get table name from file name without extension

    # Iterate through each row in the CSV file
    for index, row in df.iterrows():
        # Iterate through each column in the row
        for i, value in enumerate(row):
            # Concatenate the value with the column number and table name
            input_text = f"{table_name},column {i+1}: {value}"
            print('input text:', input_text)

            # Tokenize input texts
            tokenized_data = tokenizer(input_text, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

            # Prepare input tensors
            input_ids = tokenized_data['input_ids'].to(device)
            attention_mask = tokenized_data['attention_mask'].to(device)

            # Perform inference
            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask)
                last_hidden_state = outputs.hidden_states[-1]
                pooled_output = last_hidden_state[:, 0, :]
                logits = outputs.logits
                num_columns_output = torch.sigmoid(num_columns_output_layer(pooled_output))
                constraint_type_output = torch.softmax(constraint_type_output_layer(pooled_output), dim=1)

                # Add prediction for data type
                data_type_output = data_type_output_layer(pooled_output)  # Assuming you have a separate output layer for data type
                predicted_data_type = torch.argmax(data_type_output, dim=1).cpu().numpy()

            # Post-process the output
            predicted_num_columns = torch.argmax(num_columns_output, dim=1).cpu().numpy()
            predicted_constraint_types = torch.argmax(constraint_type_output, dim=1).cpu().numpy()

            # Append the predictions to the list
            predictions.append({
                'Input Text': input_text,
                'Predicted Number of Columns': predicted_num_columns[0],
                'Predicted Constraint Type': predicted_constraint_types[0],
                'Predicted Data Type': predicted_data_type[0]  # Include the predicted data type
            })

    # Convert the list of dictionaries to a DataFrame
    predictions_df_WH_dept = pd.DataFrame(predictions)

    # Save the predictions to a CSV file
    output_file_path = "/content/drive/MyDrive/Colab Notebooks/predictions_col_dept_WH.csv"
    predictions_df_WH_dept.to_csv(output_file_path, index=False)

else:
    print("Input file is not a CSV file.")

input text: department,column 1: 2
input text: department,column 2: Tool Design
input text: department,column 3: Research and Development
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 3
input text: department,column 2: Sales
input text: department,column 3: Sales and Marketing
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 4
input text: department,column 2: Marketing
input text: department,column 3: Sales and Marketing
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 5
input text: department,column 2: Purchasing
input text: department,column 3: Inventory Management
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 6
input text: department,column 2: Research and Development
input text: department,column 3: Research and Development
input text: department,column 4: 2008-04-30 00:00:00
input text: department,column 1: 7
input text: departme

In [26]:
    predictions_df_WH_dept

,Input Text,Predicted Number of Columns,Predicted Constraint Type,Predicted Data Type
0,"department,column 1: 2",1,0,3
1,"department,column 2: Tool Design",1,2,3
2,"department,column 3: Research and Development",1,2,3
3,"department,column 4: 2008-04-30 00:00:00",1,2,3
4,"department,column 1: 3",1,0,3
5,"department,column 2: Sales",1,2,3
6,"department,column 3: Sales and Marketing",1,2,3
7,"department,column 4: 2008-04-30 00:00:00",1,2,3
8,"department,column 1: 4",1,0,3
9,"department,column 2: Marketing",1,2,3


predictions_df_H_salesperson

In [32]:
import os
import pandas as pd

# Specify the path to the CSV file
input_file_path = "/content/drive/MyDrive/Colab Notebooks/Data_subset_adventures_headers/salesperson.csv"


# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your pre-trained BERT model and tokenizer
model_path = "/content/drive/MyDrive/Colab Notebooks/Saved_Model_tablecol"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path).to(device)

# Check if the input file is a CSV file
if input_file_path.endswith('.csv'):
    # Initialize a list to store the predictions
    predictions = []

    # Read the CSV file
    df = pd.read_csv(input_file_path)  # Read CSV without header
    table_name = os.path.splitext(os.path.basename(input_file_path))[0]  # Get table name from file name without extension

    # Iterate through each row in the CSV file
    for index, row in df.iterrows():
        # Iterate through each column in the row
        for i, value in enumerate(row):
            # Concatenate the value with the column number and table name
            input_text = f"{table_name},column {i+1}: {value}"
            print('input text:', input_text)

            # Tokenize input texts
            tokenized_data = tokenizer(input_text, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

            # Prepare input tensors
            input_ids = tokenized_data['input_ids'].to(device)
            attention_mask = tokenized_data['attention_mask'].to(device)

            # Perform inference
            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask)
                last_hidden_state = outputs.hidden_states[-1]
                pooled_output = last_hidden_state[:, 0, :]
                logits = outputs.logits
                num_columns_output = torch.sigmoid(num_columns_output_layer(pooled_output))
                constraint_type_output = torch.softmax(constraint_type_output_layer(pooled_output), dim=1)

                # Add prediction for data type
                data_type_output = data_type_output_layer(pooled_output)  # Assuming you have a separate output layer for data type
                predicted_data_type = torch.argmax(data_type_output, dim=1).cpu().numpy()

            # Post-process the output
            predicted_num_columns = torch.argmax(num_columns_output, dim=1).cpu().numpy()
            predicted_constraint_types = torch.argmax(constraint_type_output, dim=1).cpu().numpy()

            # Append the predictions to the list
            predictions.append({
                'Input Text': input_text,
                'Predicted Number of Columns': predicted_num_columns[0],
                'Predicted Constraint Type': predicted_constraint_types[0],
                'Predicted Data Type': predicted_data_type[0]  # Include the predicted data type
            })

    # Convert the list of dictionaries to a DataFrame
    predictions_df_H_salesperson = pd.DataFrame(predictions)

    # Save the predictions to a CSV file
    output_file_path = "/content/drive/MyDrive/Colab Notebooks/predictions_df_H_salesperson.csv"
    predictions_df_H_salesperson.to_csv(output_file_path, index=False)

else:
    print("Input file is not a CSV file.")


input text: salesperson,column 1: 274
input text: salesperson,column 2: nan
input text: salesperson,column 3: nan
input text: salesperson,column 4: 0
input text: salesperson,column 5: 0.0
input text: salesperson,column 6: 559697.5639
input text: salesperson,column 7: 0.0
input text: salesperson,column 8: 48754992-9ee0-4c0e-8c94-9451604e3e02
input text: salesperson,column 9: 2010-12-28 00:00:00
input text: salesperson,column 1: 275
input text: salesperson,column 2: 2.0
input text: salesperson,column 3: 300000.0
input text: salesperson,column 4: 4100
input text: salesperson,column 5: 0.012
input text: salesperson,column 6: 3763178.1787
input text: salesperson,column 7: 1750406.4785
input text: salesperson,column 8: 1e0a7274-3064-4f58-88ee-4c6586c87169
input text: salesperson,column 9: 2011-05-24 00:00:00
input text: salesperson,column 1: 276
input text: salesperson,column 2: 4.0
input text: salesperson,column 3: 250000.0
input text: salesperson,column 4: 2000
input text: salesperson,colu

In [33]:
predictions_df_H_salesperson

,Input Text,Predicted Number of Columns,Predicted Constraint Type,Predicted Data Type
0,"salesperson,column 1: 274",1,2,3
1,"salesperson,column 2: nan",1,0,0
2,"salesperson,column 3: nan",1,0,0
3,"salesperson,column 4: 0",1,2,0
4,"salesperson,column 5: 0.0",1,2,3
...,...,...,...,...
148,"salesperson,column 5: 0.016",1,2,3
149,"salesperson,column 6: 3121616.3202",1,0,3
150,"salesperson,column 7: 2396539.7601",1,0,3
151,"salesperson,column 8: f509e3d4-76c8-42aa-b353-...",1,2,3


In [29]:
import os
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Define the directory where the model and tokenizer are saved
output_dir = "/content/drive/MyDrive/Colab Notebooks/Saved_Model_tablecol"

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Load the model
model = BertForSequenceClassification.from_pretrained(output_dir)

# Define the path to the CSV file
input_file_path = "/content/drive/MyDrive/Colab Notebooks/data_subset_adventureworks/employee.csv"

# Check if the input file is a CSV file
if input_file_path.endswith('.csv'):
    # Initialize a list to store the predictions
    predictions = []

    # Read the CSV file
    df = pd.read_csv(input_file_path, header=None)
    table_name = os.path.splitext(os.path.basename(input_file_path))[0]

    # Iterate through each row in the CSV file
    for index, row in df.iterrows():
        # Initialize a list to store the tokenized values
        tokenized_values = []

        # Iterate through each column in the row
        for col_name, val in zip(df.columns, row):
            # Concatenate the table name with each column value and tokenize it
            input_text = f"{table_name}: {col_name} {val}"
            tokenized_val = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt")
            tokenized_values.append(tokenized_val.input_ids)

        # Concatenate the tokenized values along with the table name
        tokenized_input = torch.cat(tokenized_values, dim=1)

        # Perform inference using the pre-trained BERT model
        with torch.no_grad():
            outputs = model(tokenized_input)

        # Process the model outputs (e.g., get predicted labels)
        # Add the predictions to the list
        predictions.append({
            'Input Text': input_text,
            'Predicted Number of Columns': outputs.logits[0][0].argmax().item(),
            'Predicted Constraint Type': outputs.logits[0][1].argmax().item(),
            'Predicted Data Type': outputs.logits[0][2].argmax().item()
        })

    # Convert the predictions list to a DataFrame
    predictions_df_emp = pd.DataFrame(predictions)

    # Save the predictions DataFrame to a CSV file
    output_csv_path = "/content/drive/MyDrive/Colab Notebooks/Predictions/predictions_employee.csv"
    predictions_df_emp.to_csv(output_csv_path, index=False)
